In [1]:
import pandas as pd
from datetime import date, timedelta
import time 
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
def aver():
    x = 6
    j =  "asdasd"
    return x, j

In [6]:
a, b = aver()

In [12]:
print(a,b)

6 asdasd


In [2]:
def send_email(subject, body, sender, recipients, password):
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)

    # Record the MIME types of both parts - text/plain and text/html.
    part2 = MIMEText(body, 'html')  
    msg.attach(part2)

    smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp_server.login(sender, password)
    smtp_server.sendmail(sender, recipients, msg.as_string())
    smtp_server.quit()


In [3]:
df = pd.read_csv('oportunidades.csv', encoding='utf-8')

In [4]:
today = date.today().strftime("%d/%m/%Y")
today

'26/03/2023'

In [ ]:
print(f"""
{len( df[['url_detail_id','title']][ (df['scrapped_day'] == today) & (df['is_alert'] == True)] )} \
alertas de \
{len( df[['url_detail_id','title']][ (df['scrapped_day'] == today)] )} oportunidades nuevas
""")

In [ ]:
df[['url_detail_id','title']][ (df['scrapped_day'] == today) & (df['is_alert'] == True) ]

### por fechas

#### por día

In [ ]:
#selecciona las oportunidades escrapeadas hoy y que tengan alerta
nuevas_alertas = df[['url_detail_id','title','source','location','opening_date','closing_date']]\
    [
        (df['scrapped_day'] == today) & (df['is_alert'] == True) 
    ]
nuevas_alertas

In [ ]:
#selecciona las oportunidades escrapeadas hoy
nuevas_oportunidades = df[['url_detail_id','title']]\
    [
        (df['scrapped_day'] == today)
    ]
nuevas_oportunidades

In [8]:
print(f"""
{len(nuevas_alertas)} \
alertas de \
{len(nuevas_oportunidades)} oportunidades nuevas
""")


2 alertas de 78 oportunidades nuevas



In [24]:
html = """
    <html>
    <head></head>
    <body>
        <h2>Se encontraron nuevas oportunidades!!::</h2><br><br>
    """

# iterate over each group
for group_name, df_group in nuevas_alertas.groupby('source'):
    html += f"<h3>{group_name}</h3>"
    i = 1
    for row_index, row in df_group.iterrows():
        detail_url = row['url_detail_id'] 
        title = row['title'][0:85] + "..." 
        location = row['location'] if row['location'] else False
        op_date = row['opening_date']
        cl_date = row['closing_date']

        html += f"{i}. <a href='{detail_url}'>{title}</a><br>"
        
        html += f"Deadline: {cl_date}<br>" if cl_date else ""
        html += f"Fecha de publicacion: {op_date}<br>" if op_date else ""
        html += f"Ubicación: {location}<br><br>" if location else "<br><br>"

        i+=1

html += """
    </p>
    </body>
    </html>
    """

print(html)


    <html>
    <head></head>
    <body>
        <h2>Se encontraron nuevas oportunidades!!::</h2><br><br>
    <h3>GIZ</h3>1. <a href='https://jobs.giz.de/index.php?ac=jobad&id=66559'>Praktikant*in „Unterstützung des Global Health Protection Programme“...</a><br>Deadline: 2023-03-27<br>Fecha de publicacion: 2023-03-13<br>Ubicación: Berlin<br><br>2. <a href='https://jobs.giz.de/index.php?ac=jobad&id=66494'>Berater*in im Bereich Globale Gesundheit...</a><br>Deadline: 2023-03-26<br>Fecha de publicacion: 2023-03-10<br>Ubicación: Bonn<br><br><h3>unops_interns</h3>1. <a href='https://jobs.unops.org/Pages/ViewVacancy/VADetails.aspx?id=25634'>Pasante de Igualdad de Género (Gestión del Conocimiento)...</a><br>Deadline: 09-Apr-2023<br>Fecha de publicacion: 15-Mar-2023<br>Ubicación: Ciudad de Guatemala, Guatemala<br><br>
    </p>
    </body>
    </html>
    


In [71]:
#html prototipo,--DESCONTINUADO
html = """
    <html>
    <head></head>
    <body>
        <p>Se encontraron nuevas oportunidades!!::<br><br>
    """

for index, row in nuevas_alertas.iterrows():
    detail_url = row['url_detail_id'] 
    title = row['title'][0:85] + "..." 
    html += f"""
    <a href="{detail_url}">{title}</a><br>
    """

html += """
    </p>
    </body>
    </html>
    """

In [72]:
subject = "Nuevas oportunidades"
body = html
sender = "garciaamayajuancristobal.soft@gmail.com"
recipients = ["iswjuanamaya@gmail.com"]
password = "bfqqofvsrmtefbbo"

send_email(subject, body, sender, recipients, password)

#### por mes

In [15]:
#get the month of today
month = date.today().strftime("%m").replace("0","")
month

'3'

In [16]:
df['month'] = pd.to_datetime(df['scrapped_day'], dayfirst=True).dt.month

In [17]:
#selecciona las oportunidades que tengan alerta y hallan sido escrapeadas el mes actual
df[
    (df['month'] == int(month)) & (df['is_alert'] == True) 
]

,url_detail_id,scrapped_day,title,opening_date,closing_date,source,is_alert,location,tipo,reference,organization,month
18,https://jobs.giz.de/index.php?ac=jobad&id=66420,19/03/2023,Spezialist*in für Vergabe und fachliche/method...,2023-03-16,2023-04-16,GIZ,True,Eschborn,NaN,NaN,NaN,3
24,https://jobs.giz.de/index.php?ac=jobad&id=66612,19/03/2023,Junior- Advisor (m/f/d) for demand-side subsid...,2023-03-15,2023-03-22,GIZ,True,Bonn,NaN,NaN,NaN,3
44,https://jobs.giz.de/index.php?ac=jobad&id=66559,19/03/2023,Praktikant*in „Unterstützung des Global Health...,2023-03-13,2023-03-27,GIZ,True,Berlin,NaN,NaN,NaN,3
54,https://iadbcareers.referrals.selectminds.com/...,19/03/2023,IDB Invest – Investment Operations Consultant ...,"Mar 15, 2023","Mar 29, 2023",IDB,True,Peru - Country Office,NaN,NaN,NaN,3
61,https://paho.wd5.myworkdayjobs.com/en-US/pahoc...,19/03/2023,PAHO Consultant - Determinantes Ambientales de...,2023-03-14,"marzo 31, 2023, 11:59 PM Colombia Standard Time",PAHO,True,Peru,NaN,NaN,NaN,3
64,https://paho.wd5.myworkdayjobs.com/en-US/pahoc...,19/03/2023,PAHO Consultant - SF Technical Researcher/Writ...,2023-03-13,"March 30, 2023, 11:59 PM Eastern Time",PAHO,True,United States of America,NaN,NaN,NaN,3
65,https://paho.wd5.myworkdayjobs.com/en-US/pahoc...,19/03/2023,PAHO Consultant - Strategic Communications Con...,2023-03-13,"March 30, 2023, 11:59 PM Eastern Time",PAHO,True,United States of America,NaN,NaN,NaN,3
75,https://www.ungm.org/Public/Notice/195205,19/03/2023,Engineering Design Services for Rehabilitation...,19-Mar-2023,31-Mar-2023 09:00 (GMT 0.00),ungm,True,Yemen,Request for quotation,NaN,NaN,3
102,https://jobs.unops.org/Pages/ViewVacancy/VADet...,19/03/2023,Programme Manager,17-Mar-2023,06-Apr-2023,unops,True,"Bangkok, Thailand",NaN,NaN,NaN,3
105,https://jobs.unops.org/Pages/ViewVacancy/VADet...,19/03/2023,Senior Project Manager,16-Mar-2023,04-Apr-2023,unops,True,"Tunis, Tunisia",NaN,NaN,NaN,3


In [18]:
#comprueba que seleccione todos los del mes
df[
    (df['is_alert'] == True) 
]

,url_detail_id,scrapped_day,title,opening_date,closing_date,source,is_alert,location,tipo,reference,organization,month
18,https://jobs.giz.de/index.php?ac=jobad&id=66420,19/03/2023,Spezialist*in für Vergabe und fachliche/method...,2023-03-16,2023-04-16,GIZ,True,Eschborn,NaN,NaN,NaN,3
24,https://jobs.giz.de/index.php?ac=jobad&id=66612,19/03/2023,Junior- Advisor (m/f/d) for demand-side subsid...,2023-03-15,2023-03-22,GIZ,True,Bonn,NaN,NaN,NaN,3
44,https://jobs.giz.de/index.php?ac=jobad&id=66559,19/03/2023,Praktikant*in „Unterstützung des Global Health...,2023-03-13,2023-03-27,GIZ,True,Berlin,NaN,NaN,NaN,3
54,https://iadbcareers.referrals.selectminds.com/...,19/03/2023,IDB Invest – Investment Operations Consultant ...,"Mar 15, 2023","Mar 29, 2023",IDB,True,Peru - Country Office,NaN,NaN,NaN,3
61,https://paho.wd5.myworkdayjobs.com/en-US/pahoc...,19/03/2023,PAHO Consultant - Determinantes Ambientales de...,2023-03-14,"marzo 31, 2023, 11:59 PM Colombia Standard Time",PAHO,True,Peru,NaN,NaN,NaN,3
64,https://paho.wd5.myworkdayjobs.com/en-US/pahoc...,19/03/2023,PAHO Consultant - SF Technical Researcher/Writ...,2023-03-13,"March 30, 2023, 11:59 PM Eastern Time",PAHO,True,United States of America,NaN,NaN,NaN,3
65,https://paho.wd5.myworkdayjobs.com/en-US/pahoc...,19/03/2023,PAHO Consultant - Strategic Communications Con...,2023-03-13,"March 30, 2023, 11:59 PM Eastern Time",PAHO,True,United States of America,NaN,NaN,NaN,3
75,https://www.ungm.org/Public/Notice/195205,19/03/2023,Engineering Design Services for Rehabilitation...,19-Mar-2023,31-Mar-2023 09:00 (GMT 0.00),ungm,True,Yemen,Request for quotation,NaN,NaN,3
102,https://jobs.unops.org/Pages/ViewVacancy/VADet...,19/03/2023,Programme Manager,17-Mar-2023,06-Apr-2023,unops,True,"Bangkok, Thailand",NaN,NaN,NaN,3
105,https://jobs.unops.org/Pages/ViewVacancy/VADet...,19/03/2023,Senior Project Manager,16-Mar-2023,04-Apr-2023,unops,True,"Tunis, Tunisia",NaN,NaN,NaN,3


#### para los lunes

In [19]:
#genera datetime del día actual con python,
today = date.today()
# detecta si es lunes
if today.weekday() == 0:
    print("es lunes")


In [20]:
today = date.today()
if today.weekday() == 0:
    print("es lunes")
sab_dom_lun = [
    today.strftime("%d/%m/%Y"),
    (today - timedelta(1)).strftime("%d/%m/%Y"), 
    (today - timedelta(2)).strftime("%d/%m/%Y") 
    ]

sab_dom_lun

['22/03/2023', '21/03/2023', '20/03/2023']

In [ ]:
df[
    (df['scrapped_day'].isin(sab_dom_lun)) & (df['is_alert'] == True)
]

#### semanal

In [4]:
today = date.today()
fechas_semanales = [
    today.strftime("%d/%m/%Y"),
    (today - timedelta(1)).strftime("%d/%m/%Y"), 
    (today - timedelta(2)).strftime("%d/%m/%Y"),
    (today - timedelta(3)).strftime("%d/%m/%Y"),
    (today - timedelta(4)).strftime("%d/%m/%Y"),
    (today - timedelta(5)).strftime("%d/%m/%Y"),
    (today - timedelta(6)).strftime("%d/%m/%Y")
    ]
fechas_semanales

['28/03/2023',
 '27/03/2023',
 '26/03/2023',
 '25/03/2023',
 '24/03/2023',
 '23/03/2023',
 '22/03/2023']

In [14]:
#selecciona las oportunidades escrapeadas sabado, domingo y lunes y que tengan alerta
nuevas_alertas = df[['url_detail_id','title','source','location','opening_date','closing_date']]\
    [
        (df['scrapped_day'].isin(fechas_semanales)) & (df['is_alert'] == True)
    ]
#selecciona las oportunidades escrapeadas sabado, domingo y lunes 
nuevas_oportunidades = df[['url_detail_id','title']]\
                        [
                            (df['scrapped_day'].isin(fechas_semanales)) 
                        ]
msg = f""" {len(nuevas_alertas)} \
alertas detectadas en {len(nuevas_oportunidades)} oportunidades recolectadas"""

print(msg)

 48 alertas detectadas en 651 oportunidades recolectadas


In [9]:
organizaciones = {
    'IDB':'Banco Interamericano de Desarrollo',
    'GIZ':'GIZ',
    'wbg':'World Bank',
    'PAHO':'PAHO',
    'OECD':'OECD u OCDE',
    'unops':'UNOPS',
    'ungm':'Naciones Unidas'
    }

In [15]:
html = f"""
    <html>
    <head></head>
    <body>
        <h2>{msg}</h2>
    """

# iterate over each group
for group_name, df_group in nuevas_alertas.groupby('source'):
    html += f"<h3>{organizaciones[group_name]}</h3>"
    i = 1
    for row_index, row in df_group.iterrows():
        detail_url = row['url_detail_id'] 
        title = row['title'][0:85] + "..." 
        location = row['location'] if row['location'] else False
        op_date = row['opening_date']
        cl_date = row['closing_date']

        html += f"{i}. <a href='{detail_url}'>{title}</a><br>"
        
        html += f"Deadline: {cl_date}<br>" if cl_date else ""
        html += f"Fecha de publicacion: {op_date}<br>" if op_date else ""
        html += f"Ubicación: {location}<br><br>" if location else "<br><br>"

        i+=1

html += """
    </p>
    </body>
    </html>
    """

print(html)


    <html>
    <head></head>
    <body>
        <h2> 48 alertas detectadas en 651 oportunidades recolectadas</h2>
    <h3>GIZ</h3>1. <a href='https://jobs.giz.de/index.php?ac=jobad&id=66311'>Fachkraft im Zivilen Friedensdienst (ZFD) für psychosoziale Beratung...</a><br>Deadline: 2023-04-19<br>Fecha de publicacion: 2023-03-23<br>Ubicación: Guatemala Stadt<br><br>2. <a href='https://jobs.giz.de/index.php?ac=jobad&id=66685'>Internship with Energising Development (EnDev) in Malawi...</a><br>Deadline: 2023-04-04<br>Fecha de publicacion: 2023-03-21<br>Ubicación: Lilongwe<br><br>3. <a href='https://jobs.giz.de/index.php?ac=jobad&id=66183'>Development worker (m/f/d) in the field of Sanitary, Heating and Climate Technology...</a><br>Deadline: 2023-04-04<br>Fecha de publicacion: 2023-03-21<br>Ubicación: Trang Bom<br><br>4. <a href='https://jobs.giz.de/index.php?ac=jobad&id=66559'>Praktikant*in „Unterstützung des Global Health Protection Programme“...</a><br>Deadline: 2023-03-27<br>Fecha de publ

In [11]:
subject = "oportunidades semanales"
body = html
sender = "alquimiadigital23@gmail.com"
recipients = ["iswjuanamaya@gmail.com", "gustavo.gilramos@gmail.com"]
password = "pyfiewjrnjmqtrnl"

send_email(subject, body, sender, recipients, password)